In [2]:
import tensorflow as tf
import numpy as np

amplifier = 1
threshold_percentage = 0

# Define the CustomAdam optimizer class
from tensorflow.keras.optimizers import Adam

@tf.keras.utils.register_keras_serializable()
class CustomAdam(Adam):
    pass

# Load the pre-trained model
model = tf.keras.models.load_model('MAG_MNIST_vgg16_0&1_2&3.h5', custom_objects={'CustomAdam': CustomAdam})

# Load the additional pre-trained model for weight multiplication
progress_model = tf.keras.models.load_model('freezing_value_progress_MNIST_0&1_2&3.h5', custom_objects={'CustomAdam': CustomAdam})

# Copy the structure and parameters of the model
freezing_value_mnist = tf.keras.models.clone_model(model)
freezing_value_mnist.set_weights(model.get_weights())

# Multiply the weights of the cloned model with the weights of the progress model
for layer, progress_layer in zip(freezing_value_mnist.layers, progress_model.layers):
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        progress_weights = progress_layer.get_weights()
        new_weights = []

        for weight_matrix, progress_weight_matrix in zip(weights, progress_weights):
            new_weight_matrix = weight_matrix * progress_weight_matrix
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Modify the weights as specified
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Compute the biggest weight magnitude in the whole layer
            max_magnitude = np.max(np.abs(weight_matrix))
            
            # Apply the transformation to each weight
            new_weight_matrix = 1 - (np.abs(weight_matrix) / max_magnitude) ** amplifier
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Further modify the weights by setting the smallest specified percentage weights for each layer to 0
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Flatten the weight matrix and sort the weights by magnitude
            flat_weights = weight_matrix.flatten()
            threshold = np.percentile(np.abs(flat_weights), threshold_percentage)
            
            # Set the smallest weights to 0
            new_weight_matrix = np.where(np.abs(weight_matrix) < threshold, 0, weight_matrix)
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Print all the weights in the model
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        print(f"Weights for layer {layer.name}:")
        for weight_matrix in weights:
            print(weight_matrix)

# Save the modified model
freezing_value_mnist.save('freezing_value_mag_0&1_2&3_0%_^1.h5')

print("Model 'freezing_value_mag_0&1_2&3_0%_^1.h5' has been saved.")


2024-08-07 21:44:46.354995: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-07 21:44:46.371268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 21:44:46.390912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 21:44:46.396903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-07 21:44:46.411266: I tensorflow/core/platform/cpu_feature_guar

Weights for layer conv2d:
[[[[0.7188413  1.         0.35853595 ... 0.07467419 0.37707484
    0.99499613]
   [0.5386264  0.04262632 0.11386675 ... 0.0990482  1.
    0.73952544]
   [0.8644426  0.09658986 0.99325526 ... 0.4703148  0.884418
    0.9923396 ]]

  [[0.9690242  0.76453817 0.45328188 ... 0.41512668 0.8593259
    1.        ]
   [1.         0.2779475  0.25458837 ... 0.7248126  0.39767563
    0.8634795 ]
   [1.         0.74159425 0.8030417  ... 0.9172406  0.3555919
    0.70041335]]

  [[0.5379707  0.90593433 1.         ... 1.         1.
    0.58154655]
   [0.16242987 1.         0.14219952 ... 0.11222154 0.35445267
    0.63621557]
   [0.14801025 0.7916908  0.91204584 ... 0.73996    0.5258559
    1.        ]]]


 [[[0.8942907  0.6100334  0.26147628 ... 0.5346029  1.
    0.44559443]
   [1.         0.9278322  0.14848745 ... 0.9120462  1.
    0.8645911 ]
   [0.2534505  1.         0.84016705 ... 0.57345986 0.8085952
    0.19772315]]

  [[0.6371153  0.16154963 1.         ... 1.         0.

Weights for layer dense:
[[1.         0.9518087  0.98857343 ... 0.43990147 0.8878308  0.58372664]
 [0.88213074 0.91065377 1.         ... 0.7804644  0.9022013  0.7991308 ]
 [0.57414424 0.6701118  0.8406247  ... 0.9591544  0.57920325 0.6130085 ]
 ...
 [0.6254381  0.39345634 0.3492322  ... 0.95306957 0.6093788  0.5568173 ]
 [0.49592298 0.77399623 0.8414911  ... 0.37452614 0.40161222 1.        ]
 [0.40584165 0.48100632 0.6798794  ... 0.7057259  0.84773326 0.7426049 ]]
[0.8167566 0.9521167 1.        ... 0.5976616 1.        1.       ]
Weights for layer dense_1:
[[0.7364496  0.28857213 0.7536905  ... 0.8101394  1.         0.2319997 ]
 [0.34224492 0.3428461  0.26699752 ... 1.         0.76939344 1.        ]
 [1.         0.40960675 1.         ... 0.38063562 0.353109   0.9164959 ]
 ...
 [0.44928384 1.         0.5096849  ... 0.49828255 0.296144   0.5059986 ]
 [0.49184215 0.53435636 0.37814647 ... 0.25018722 0.18842858 0.14684272]
 [0.6324731  0.8161994  1.         ... 0.5255027  1.         0.30142